First updated @ Oct 31, 2021      Happy Halloween👻

Last updated @ Nov 2, 2021

In [2]:
# import library
import pandas as pd
import numpy as np

## Read all movies

- get the full movie list and the imdbid

In [9]:
df = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')

In [10]:
df.tail()

,title,imdbid,id,rating,year
9368,Love Hard,10752004,10152,2,2021
9369,Cruella,,10155,3,2021
9370,West Side Story,3581652,10157,3,2021
9371,Every Time a Bell Rings,15943414,10158,3,2021
9372,Single All The Way,14315756,10161,3,2021


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   9373 non-null   object
 1   imdbid  9373 non-null   object
 2   id      9373 non-null   int64 
 3   rating  9373 non-null   int64 
 4   year    9373 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 366.3+ KB


In [12]:
# save dataframe to a csv file
df.to_csv('Bechdel.csv')

## Get movie by imdbid

- then loop through the imdbid in the previous dataframe to get each movie info
- create a new dataframe and output the newer csv file

Note: The website is fragile and you may experience timeout error. Make sure to save dataframe to csv so that you don't lose everything.

In [3]:
pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(14315756), typ='series')

imdbid                    14315756
rating                           3
title           Single All The Way
visible                          1
date           2021-12-15 10:10:36
id                           10161
year                          2021
dubious                          0
submitterid                  19753
dtype: object

In [17]:
count = 1
# create an empty dataframe
df_new = pd.DataFrame(columns=['title', 'year', 'rating', 'dubious', 'imdbid', 'id', 'submitterid', 'date', 'visible'])

# call from the latest to the earliest movie, in case the website crashes
for imdbid in df['imdbid'][-1::-1]:
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    # print(series)
    df_new = df_new.append(series, ignore_index=True)
    # every 10 movies, we save to a csv file, 
    # so that we wouldn't lose everything when there is an error
    if count % 10 == 0:
        df_new.to_csv('Bechdel_detailed_reversed.csv')
    count += 1
df_new.to_csv('Bechdel_detailed_reversed.csv')

This took about 7 hours to execute

In [18]:
# reverse 
df_new = df_new.iloc[::-1].reset_index(drop=True)
df_new.to_csv('Bechdel_detailed.csv') #, index=False)

## Check data integrity

- check duplicates
- check row numbers
- check index

In [14]:
df = pd.read_csv('./Bechdel.csv', index_col=0)
df_to_check = pd.read_csv('./Bechdel_detailed.csv', index_col=0)
df_to_check

,title,year,rating,dubious,imdbid,id,submitterid,date,visible
0,Passage de Venus,1874.0,0.0,0.0,3155794.0,9602.0,18880.0,2021-04-02 20:58:09,1.0
1,La Rosace Magique,1877.0,0.0,0.0,14495706.0,9804.0,19145.0,2021-05-11 00:11:22,1.0
2,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,9603.0,18882.0,2021-04-03 02:25:27,1.0
3,Le singe musicien,1878.0,0.0,0.0,12592084.0,9806.0,19151.0,2021-05-11 23:38:54,1.0
4,Athlete Swinging a Pick,1881.0,0.0,0.0,7816420.0,9816.0,19162.0,2021-05-13 01:32:14,1.0
...,...,...,...,...,...,...,...,...,...
9368,Love Hard,2021.0,2.0,0.0,10752004.0,10152.0,19735.0,2021-12-05 19:22:20,1.0
9369,Cruella,2021.0,3.0,0.0,3228774.0,9861.0,19231.0,2021-06-01 03:16:58,1.0
9370,West Side Story,2021.0,3.0,0.0,3581652.0,10157.0,19743.0,2021-12-10 03:10:09,1.0
9371,Every Time a Bell Rings,2021.0,3.0,0.0,15943414.0,10158.0,19744.0,2021-12-10 08:03:02,1.0


In [6]:
def check_dataset(df_to_check):
    lengthFlag = len(df_to_check) == len(df)
    print(f'Same number of rows? {lengthFlag}, should be {len(df)} movies, got {len(df_to_check)} movies')
    if lengthFlag:
        movieFlag = True
        for i in range(len(df)):
            if df['imdbid'].iloc[i] != df_to_check['imdbid'].iloc[i]:
                print(i, df['imdbid'].iloc[i], df_to_check['imdbid'].iloc[i])
                movieFlag = False
                break
    print(f'Same movies? {movieFlag}')

In [7]:
# check integrity and fail
check_dataset(df_to_check)

Same number of rows? True, should be 9373 movies, got 9373 movies
Same movies? True


# Data Cleaning

It often turns out that some movies are "not found" on the website. Why is it happening?

In [46]:
bechdel_df = pd.read_csv('Bechdel.csv', index_col=0)
bechdel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   9373 non-null   object 
 1   imdbid  9373 non-null   float64
 2   id      9373 non-null   int64  
 3   rating  9373 non-null   int64  
 4   year    9373 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 439.4+ KB


In [32]:
bechdel_detailed_df = pd.read_csv('./Bechdel_detailed.csv')
bechdel_detailed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9369 non-null   object 
 1   year         9369 non-null   float64
 2   rating       9369 non-null   float64
 3   dubious      9070 non-null   float64
 4   imdbid       9369 non-null   float64
 5   id           9369 non-null   float64
 6   submitterid  9369 non-null   float64
 7   date         9369 non-null   object 
 8   visible      9369 non-null   float64
 9   description  4 non-null      object 
 10  status       4 non-null      float64
 11  version      4 non-null      object 
dtypes: float64(8), object(4)
memory usage: 878.8+ KB


In [33]:
# There are 4 nans
bechdel_detailed_df[bechdel_detailed_df['title'].isna()]

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,description,status,version
8374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
9133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
9362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
9369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1


After checking the data, I found that 4 movies do not have a imdbid in Bechdel.csv and therefore do not have record in Bechdel_detailed.csv.

I went to IMDB and I got the imdb ids:
- 8374: 3521164
- 9133: 8022928
- 9362: 6334354
- 9369: 3228774

We can now get the detailed info using https://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=3228774

In [42]:
bechdel_df.at[8374, 'imdbid'] = 3521164
bechdel_df.at[9133, 'imdbid'] = 8022928
bechdel_df.at[9362, 'imdbid'] = 6334354
bechdel_df.at[9369, 'imdbid'] = 3228774
bechdel_df.tail()

,title,imdbid,id,rating,year
9368,Love Hard,10752004.0,10152,2,2021
9369,Cruella,3228774.0,10155,3,2021
9370,West Side Story,3581652.0,10157,3,2021
9371,Every Time a Bell Rings,15943414.0,10158,3,2021
9372,Single All The Way,14315756.0,10161,3,2021


In [44]:
bechdel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   9373 non-null   object 
 1   imdbid  9373 non-null   float64
 2   id      9373 non-null   int64  
 3   rating  9373 non-null   int64  
 4   year    9373 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 759.4+ KB


In [45]:
bechdel_df.to_csv('Bechdel.csv')

In [49]:
bechdel_detailed_df.tail()

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,description,status,version
9368,Love Hard,2021.0,2.0,0.0,10752004.0,10152.0,19735.0,2021-12-05 19:22:20,1.0,NaN,NaN,NaN
9369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Could not find movie,404.0,v1
9370,West Side Story,2021.0,3.0,0.0,3581652.0,10157.0,19743.0,2021-12-10 03:10:09,1.0,NaN,NaN,NaN
9371,Every Time a Bell Rings,2021.0,3.0,0.0,15943414.0,10158.0,19744.0,2021-12-10 08:03:02,1.0,NaN,NaN,NaN
9372,Single All The Way,2021.0,3.0,0.0,14315756.0,10161.0,19753.0,2021-12-15 10:10:36,1.0,NaN,NaN,NaN


In [51]:
for imdbid, index in zip([3521164, 8022928, 6334354, 3228774], [8374, 9133, 9362, 9369]):
    series = pd.read_json('http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=' + str(imdbid), typ='series')
    # print(series)
    bechdel_detailed_df.loc[index] = series
bechdel_detailed_df.tail()

,title,year,rating,dubious,imdbid,id,submitterid,date,visible,description,status,version
9368,Love Hard,2021.0,2.0,0,1.0752e+07,10152.0,19735.0,2021-12-05 19:22:20,1,NaN,NaN,NaN
9369,Cruella,2021.0,3.0,0,3228774,9861.0,19231.0,2021-06-01 03:16:58,1,NaN,NaN,NaN
9370,West Side Story,2021.0,3.0,0,3.58165e+06,10157.0,19743.0,2021-12-10 03:10:09,1,NaN,NaN,NaN
9371,Every Time a Bell Rings,2021.0,3.0,0,1.59434e+07,10158.0,19744.0,2021-12-10 08:03:02,1,NaN,NaN,NaN
9372,Single All The Way,2021.0,3.0,0,1.43158e+07,10161.0,19753.0,2021-12-15 10:10:36,1,NaN,NaN,NaN


In [52]:
bechdel_detailed_df.drop(['description', 'status', 'version'], axis=1, inplace=True)
bechdel_detailed_df.tail()

,title,year,rating,dubious,imdbid,id,submitterid,date,visible
9368,Love Hard,2021.0,2.0,0,1.0752e+07,10152.0,19735.0,2021-12-05 19:22:20,1
9369,Cruella,2021.0,3.0,0,3228774,9861.0,19231.0,2021-06-01 03:16:58,1
9370,West Side Story,2021.0,3.0,0,3.58165e+06,10157.0,19743.0,2021-12-10 03:10:09,1
9371,Every Time a Bell Rings,2021.0,3.0,0,1.59434e+07,10158.0,19744.0,2021-12-10 08:03:02,1
9372,Single All The Way,2021.0,3.0,0,1.43158e+07,10161.0,19753.0,2021-12-15 10:10:36,1


In [53]:
bechdel_detailed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9373 non-null   object 
 1   year         9373 non-null   float64
 2   rating       9373 non-null   float64
 3   dubious      9074 non-null   object 
 4   imdbid       9373 non-null   object 
 5   id           9373 non-null   float64
 6   submitterid  9373 non-null   float64
 7   date         9373 non-null   object 
 8   visible      9373 non-null   object 
dtypes: float64(4), object(5)
memory usage: 659.2+ KB


In [55]:
bechdel_detailed_df.to_csv('Bechdel_detailed.csv')